<a href="https://colab.research.google.com/github/Shayan1601/EX2_repo/blob/main/mountaincarcontinuous.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras
!pip install keras-rl2
!pip install gym[classic_control]
!pip install 
!pip install xvfbwrapper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gym 
import random
from rl.agents import DDPGAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
import numpy as np
from pyvirtualdisplay import Display
!apt-get install -y xvfb ffmpeg > /dev/null 2>&1
!pip install pyvirtualdisplay > /dev/null 2>&1


env = gym.make('MountainCarContinuous-v0')

states = env.observation_space.shape[0]
actions = env.action_space.shape[0]


/usr/local/lib/python3.8/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.8/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
import matplotlib.pyplot as plt

def render_as_image(env):
    plt.imshow(env.render(mode='rgb_array'))
    plt.axis('off')
    plt.show()
    render_as_image(env)

In [ ]:

import base64
import io
from IPython import display as ipythondisplay

def embed_video(video_path, width=640):
    content = io.open(video_path, 'rb').read()
    b64 = base64.b64encode(content)
    video_tag = '''
        
    '''.format(b64.decode(), width)
    ipythondisplay.display(ipythondisplay.HTML(video_tag))

In [ ]:
import imageio
from datetime import datetime

def random_filename():
    return datetime.now().strftime('%Y_%m_%d_%H_%M_%S.mp4')

class VideoRecorder:
    def __init__(self, filename=random_filename(), fps=30):
        self.filename = filename
        self.writer = imageio.get_writer(filename, fps=fps)

    def record_frame(self, env):
        frame = env.render(mode='rgb_array')
        self.writer.append_data(frame)

    def close(self, *args, **kwargs):
        self.writer.close(*args, **kwargs)

    def play(self):
        self.close()
        embed_video(self.filename)

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        self.play()

In [ ]:
!pip install imageio-ffmpeg
with VideoRecorder() as rec:
    obs = env.reset()
    while True:
        rec.record_frame(env)
        action = env.action_space.sample() 
        obs, reward, done, info = env.step(action)
        if done:
            break
     

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 24.8 MB/s eta 0:00:00


/usr/local/lib/python3.8/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


In [ ]:
from tensorflow.keras.models import Sequential
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, Concatenate
from keras.optimizers import Adam
from rl.agents import DDPGAgent
from rl.memory import SequentialMemory
from rl.random import OrnsteinUhlenbeckProcess

# Next, we build a very simple model.
actor = Sequential()
actor.add(Flatten(input_shape=(1,) + env.observation_space.shape))
actor.add(Dense(32))
actor.add(Activation('relu'))
actor.add(Dense(32))
actor.add(Activation('relu'))
actor.add(Dense(32))
actor.add(Activation('relu'))
actor.add(Dense(actions))
actor.add(Activation('tanh'))
print(actor.summary())

action_input = Input(shape=(actions,), name='action_input')
observation_input = Input(shape=(1,) + env.observation_space.shape, name='observation_input')
flattened_observation = Flatten()(observation_input)
x = Concatenate()([action_input, flattened_observation])
x = Dense(32)(x)
x = Activation('relu')(x)
x = Dense(32)(x)
x = Activation('relu')(x)
x = Dense(32)(x)
x = Activation('relu')(x)
x = Dense(1)(x)
x = Activation('linear')(x)
critic = Model(inputs=[action_input, observation_input], outputs=x)
print(critic.summary())

# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = SequentialMemory(limit=100000, window_length=1)
random_process = OrnsteinUhlenbeckProcess(size=actions, theta=.15, mu=0., sigma=.3)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_9 (Flatten)         (None, 2)                 0         
                                                                 
 dense_35 (Dense)            (None, 32)                96        
                                                                 
 activation_35 (Activation)  (None, 32)                0         
                                                                 
 dense_36 (Dense)            (None, 32)                1056      
                                                                 
 activation_36 (Activation)  (None, 32)                0         
                                                                 
 dense_37 (Dense)            (None, 32)                1056      
                                                                 
 activation_37 (Activation)  (None, 32)               

In [ ]:
agent1=DDPGAgent(nb_actions=actions, actor=actor, critic=critic, critic_action_input=action_input, memory=memory, gamma=0.95, batch_size=32,
                         nb_steps_warmup_critic=1000, nb_steps_warmup_actor=1000, train_interval=1, memory_interval=1,
                         delta_range=None, random_process=random_process, custom_model_objects={}, target_model_update=0.001)

In [ ]:
agent1.compile(Adam(lr=.0001), metrics=['mse'])

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
#for No available video device error
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"

agent1.fit(env, nb_steps=50000, visualize=True, verbose=1, nb_max_episode_steps=999)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 336s 33ms/step - reward: -0.0198
10 episodes - episode_reward: -19.546 [-78.716, 74.890] - loss: 0.017 - mse: 0.035 - mean_q: 0.041

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 331s 33ms/step - reward: -0.0264
10 episodes - episode_reward: -26.497 [-62.801, -7.356] - loss: 0.418 - mse: 0.835 - mean_q: 0.021

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 332s 33ms/step - reward: -0.0399
10 episodes - episode_reward: -38.600 [-75.736, -5.570] - loss: 0.182 - mse: 0.364 - mean_q: 0.020

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 332s 33ms/step - reward: -0.0170
10 episodes - episode_reward: -17.223 [-29.218, -6.240] - loss: 0.089 - mse: 0.179 - mean_q: 0.017

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 332s 33ms/step - reward: -0.0304


In [ ]:
scores = agent1.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: -0.900, steps: 999
Episode 2: reward: -0.893, steps: 999
Episode 3: reward: -0.850, steps: 999
Episode 4: reward: -0.898, steps: 999
Episode 5: reward: -0.901, steps: 999
Episode 6: reward: -0.896, steps: 999
Episode 7: reward: -0.809, steps: 999
Episode 8: reward: -0.758, steps: 999
Episode 9: reward: -0.839, steps: 999
Episode 10: reward: -0.901, steps: 999
Episode 11: reward: -0.894, steps: 999
Episode 12: reward: -0.890, steps: 999
Episode 13: reward: -0.796, steps: 999
Episode 14: reward: -0.901, steps: 999
Episode 15: reward: -0.902, steps: 999
Episode 16: reward: -0.872, steps: 999
Episode 17: reward: -0.900, steps: 999
Episode 18: reward: -0.902, steps: 999
Episode 19: reward: -0.791, steps: 999
Episode 20: reward: -0.901, steps: 999
Episode 21: reward: -0.899, steps: 999
Episode 22: reward: -0.742, steps: 999
Episode 23: reward: -0.827, steps: 999
Episode 24: reward: -0.798, steps: 999
Episode 25: reward: -0.834, steps: 999
Episo